## Flight Data Analysis using SQL
I will be using SQL queries to answer some questions related to the `Airlines (travel.sqlite)` dataset. 

#### The Dataset
The Dataset has been downloaded from Kaggle: [Airlines SQLite](https://www.kaggle.com/code/mpwolke/airlines-sqlite)<br>

#### The Questions
The questions have been made using Gemini and also by me. The questions I would be answering in this notebook will be:
1. Find the number of flights that arrived delayed (i.e., actual_arrival is later than scheduled_arrival) for each arrival airport (arrival_airport).
2. Popular Routes: Identify the most frequently traveled routes based on the number of entries in the ticket_flights table, considering combinations of departure_airport and arrival_airport from the flights table.
3. Top 5 Most Booked Flights
4. Top 5 Most Used Aircraft Models

In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
connection = sqlite3.connect('travel.sqlite')

In [3]:
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", connection)
tables

,type,name,tbl_name,rootpage,sql
0,table,aircrafts_data,aircrafts_data,2,CREATE TABLE aircrafts_data (\r\n aircraft_...
1,table,airports_data,airports_data,3,CREATE TABLE airports_data (\r\n airport_co...
2,table,boarding_passes,boarding_passes,4,CREATE TABLE boarding_passes (\r\n ticket_n...
3,table,bookings,bookings,5,CREATE TABLE bookings (\r\n book_ref charac...
4,table,flights,flights,6,CREATE TABLE flights (\r\n flight_id intege...
5,table,seats,seats,7,CREATE TABLE seats (\r\n aircraft_code char...
6,table,ticket_flights,ticket_flights,8,CREATE TABLE ticket_flights (\r\n ticket_no...
7,table,tickets,tickets,9,CREATE TABLE tickets (\r\n ticket_no charac...


In [19]:
tables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   type      8 non-null      object
 1   name      8 non-null      object
 2   tbl_name  8 non-null      object
 3   rootpage  8 non-null      int64 
 4   sql       8 non-null      object
dtypes: int64(1), object(4)
memory usage: 452.0+ bytes


extract individual tables

In [4]:
flights = pd.read_sql("SELECT * FROM flights;", connection)
flights

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,1185,PG0134,2017-09-10 09:50:00+03,2017-09-10 14:55:00+03,DME,BTK,Scheduled,319,\N,\N
1,3979,PG0052,2017-08-25 14:50:00+03,2017-08-25 17:35:00+03,VKO,HMA,Scheduled,CR2,\N,\N
2,4739,PG0561,2017-09-05 12:30:00+03,2017-09-05 14:15:00+03,VKO,AER,Scheduled,763,\N,\N
3,5502,PG0529,2017-09-12 09:50:00+03,2017-09-12 11:20:00+03,SVO,UFA,Scheduled,763,\N,\N
4,6938,PG0461,2017-09-04 12:25:00+03,2017-09-04 13:20:00+03,SVO,ULV,Scheduled,SU9,\N,\N
...,...,...,...,...,...,...,...,...,...,...
33116,33117,PG0063,2017-08-02 19:25:00+03,2017-08-02 20:10:00+03,SKX,SVO,Arrived,CR2,2017-08-02 19:25:00+03,2017-08-02 20:10:00+03
33117,33118,PG0063,2017-07-28 19:25:00+03,2017-07-28 20:10:00+03,SKX,SVO,Arrived,CR2,2017-07-28 19:30:00+03,2017-07-28 20:15:00+03
33118,33119,PG0063,2017-09-08 19:25:00+03,2017-09-08 20:10:00+03,SKX,SVO,Scheduled,CR2,\N,\N
33119,33120,PG0063,2017-08-01 19:25:00+03,2017-08-01 20:10:00+03,SKX,SVO,Arrived,CR2,2017-08-01 19:26:00+03,2017-08-01 20:12:00+03


In [5]:
aircrafts_data = pd.read_sql("SELECT * FROM aircrafts_data;", connection)
aircrafts_data

,aircraft_code,model,range
0,773,"{""en"": ""Boeing 777-300"", ""ru"": ""Боинг 777-300""}",11100
1,763,"{""en"": ""Boeing 767-300"", ""ru"": ""Боинг 767-300""}",7900
2,SU9,"{""en"": ""Sukhoi Superjet-100"", ""ru"": ""Сухой Суп...",3000
3,320,"{""en"": ""Airbus A320-200"", ""ru"": ""Аэробус A320-...",5700
4,321,"{""en"": ""Airbus A321-200"", ""ru"": ""Аэробус A321-...",5600
5,319,"{""en"": ""Airbus A319-100"", ""ru"": ""Аэробус A319-...",6700
6,733,"{""en"": ""Boeing 737-300"", ""ru"": ""Боинг 737-300""}",4200
7,CN1,"{""en"": ""Cessna 208 Caravan"", ""ru"": ""Сессна 208...",1200
8,CR2,"{""en"": ""Bombardier CRJ-200"", ""ru"": ""Бомбардье ...",2700


In [6]:
airports_data = pd.read_sql("SELECT * FROM airports_data;", connection)
airports_data

,airport_code,airport_name,city,coordinates,timezone
0,YKS,"{""en"": ""Yakutsk Airport"", ""ru"": ""Якутск""}","{""en"": ""Yakutsk"", ""ru"": ""Якутск""}","(129.77099609375,62.0932998657226562)",Asia/Yakutsk
1,MJZ,"{""en"": ""Mirny Airport"", ""ru"": ""Мирный""}","{""en"": ""Mirnyj"", ""ru"": ""Мирный""}","(114.03900146484375,62.534698486328125)",Asia/Yakutsk
2,KHV,"{""en"": ""Khabarovsk-Novy Airport"", ""ru"": ""Хабар...","{""en"": ""Khabarovsk"", ""ru"": ""Хабаровск""}","(135.18800354004,48.5279998779300001)",Asia/Vladivostok
3,PKC,"{""en"": ""Yelizovo Airport"", ""ru"": ""Елизово""}","{""en"": ""Petropavlovsk"", ""ru"": ""Петропавловск-К...","(158.453994750976562,53.1679000854492188)",Asia/Kamchatka
4,UUS,"{""en"": ""Yuzhno-Sakhalinsk Airport"", ""ru"": ""Хом...","{""en"": ""Yuzhno-Sakhalinsk"", ""ru"": ""Южно-Сахали...","(142.718002319335938,46.8886985778808594)",Asia/Sakhalin
...,...,...,...,...,...
99,MMK,"{""en"": ""Murmansk Airport"", ""ru"": ""Мурманск""}","{""en"": ""Murmansk"", ""ru"": ""Мурманск""}","(32.7508010864257812,68.7817001342773438)",Europe/Moscow
100,ABA,"{""en"": ""Abakan Airport"", ""ru"": ""Абакан""}","{""en"": ""Abakan"", ""ru"": ""Абакан""}","(91.3850021362304688,53.7400016784667969)",Asia/Krasnoyarsk
101,BAX,"{""en"": ""Barnaul Airport"", ""ru"": ""Барнаул""}","{""en"": ""Barnaul"", ""ru"": ""Барнаул""}","(83.5384979248046875,53.363800048828125)",Asia/Krasnoyarsk
102,AAQ,"{""en"": ""Anapa Vityazevo Airport"", ""ru"": ""Витяз...","{""en"": ""Anapa"", ""ru"": ""Анапа""}","(37.3473014831539984,45.002101898192997)",Europe/Moscow


In [9]:
boarding_passes = pd.read_sql("SELECT * FROM boarding_passes;", connection)
boarding_passes

,ticket_no,flight_id,boarding_no,seat_no
0,0005435212351,30625,1,2D
1,0005435212386,30625,2,3G
2,0005435212381,30625,3,4H
3,0005432211370,30625,4,5D
4,0005435212357,30625,5,11A
...,...,...,...,...
579681,0005434302871,19945,85,20F
579682,0005432892791,19945,86,21C
579683,0005434302869,19945,87,20E
579684,0005432802476,19945,88,21F


In [8]:
bookings = pd.read_sql("SELECT * FROM bookings;", connection)
bookings

,book_ref,book_date,total_amount
0,00000F,2017-07-05 03:12:00+03,265700
1,000012,2017-07-14 09:02:00+03,37900
2,000068,2017-08-15 14:27:00+03,18100
3,000181,2017-08-10 13:28:00+03,131800
4,0002D8,2017-08-07 21:40:00+03,23600
...,...,...,...
262783,FFFEF3,2017-07-17 07:23:00+03,56000
262784,FFFF2C,2017-08-08 05:55:00+03,10800
262785,FFFF43,2017-07-20 20:42:00+03,78500
262786,FFFFA8,2017-08-08 04:45:00+03,28800


In [13]:
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262788 entries, 0 to 262787
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   book_ref      262788 non-null  object
 1   book_date     262788 non-null  object
 2   total_amount  262788 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 6.0+ MB


In [15]:
bookings.book_date = pd.to_datetime(bookings.book_date)
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262788 entries, 0 to 262787
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype                    
---  ------        --------------   -----                    
 0   book_ref      262788 non-null  object                   
 1   book_date     262788 non-null  datetime64[ns, UTC+03:00]
 2   total_amount  262788 non-null  int64                    
dtypes: datetime64[ns, UTC+03:00](1), int64(1), object(1)
memory usage: 6.0+ MB


In [10]:
seats = pd.read_sql("SELECT * FROM seats;", connection)
seats

,aircraft_code,seat_no,fare_conditions
0,319,2A,Business
1,319,2C,Business
2,319,2D,Business
3,319,2F,Business
4,319,3A,Business
...,...,...,...
1334,773,48H,Economy
1335,773,48K,Economy
1336,773,49A,Economy
1337,773,49C,Economy


In [11]:
tickets = pd.read_sql("SELECT * FROM tickets;", connection)
tickets

,ticket_no,book_ref,passenger_id
0,0005432000987,06B046,8149 604011
1,0005432000988,06B046,8499 420203
2,0005432000989,E170C3,1011 752484
3,0005432000990,E170C3,4849 400049
4,0005432000991,F313DD,6615 976589
...,...,...,...
366728,0005435999869,D730BA,0474 690760
366729,0005435999870,D730BA,6535 751108
366730,0005435999871,A1AD46,1596 156448
366731,0005435999872,7B6A53,9374 822707


In [12]:
ticket_flights = pd.read_sql("SELECT * FROM ticket_flights;", connection)
ticket_flights

,ticket_no,flight_id,fare_conditions,amount
0,0005432159776,30625,Business,42100
1,0005435212351,30625,Business,42100
2,0005435212386,30625,Business,42100
3,0005435212381,30625,Business,42100
4,0005432211370,30625,Business,42100
...,...,...,...,...
1045721,0005435097522,32094,Economy,5200
1045722,0005435097521,32094,Economy,5200
1045723,0005435104384,32094,Economy,5200
1045724,0005435104352,32094,Economy,5200


#### Q1.

In [28]:
query = """
SELECT arrival_airport, COUNT(*) AS num_delayed_flights
FROM flights
WHERE actual_arrival > scheduled_arrival
GROUP BY arrival_airport;
"""

delayed_flights_per_airport = pd.read_sql(query, connection)

sor = delayed_flights_per_airport.sort_values(by="num_delayed_flights", ascending=False)
sor.head(5)

,arrival_airport,num_delayed_flights
13,DME,3038
80,SVO,2833
42,LED,1795
98,VKO,1629
63,OVB,1009


In [32]:
query = """
SELECT f.departure_airport, f.arrival_airport, COUNT(*) AS num_flights
FROM ticket_flights AS tf
JOIN flights AS f ON tf.flight_id = f.flight_id
GROUP BY f.departure_airport, f.arrival_airport
ORDER BY num_flights DESC;
"""

popular_routes = pd.read_sql(query, connection)

# Print the result (departure_airport, arrival_airport, and count of flights)
(popular_routes.head(5))


,departure_airport,arrival_airport,num_flights
0,SVO,LED,16461
1,DME,OVB,15903
2,SVO,SVX,15879
3,SVX,SVO,15433
4,LED,SVO,15424


In [57]:
import pandas as pd

# Assuming your connection to the database is established and assigned to 'conn'

query = """
SELECT f.flight_no, f.departure_airport, f.arrival_airport, COUNT(*) AS num_bookings
FROM ticket_flights AS tf
JOIN flights AS f ON tf.flight_id = f.flight_id
GROUP BY f.flight_no, f.departure_airport, f.arrival_airport
ORDER BY num_bookings DESC
LIMIT 5;
"""

popular_flights = pd.read_sql(query, connection)

print("Top 5 Most Booked Flights:")
(popular_flights)


Top 5 Most Booked Flights:


,flight_no,departure_airport,arrival_airport,num_bookings
0,PG0222,DME,OVB,15903
1,PG0225,SVO,SVX,15879
2,PG0226,SVX,SVO,15433
3,PG0223,OVB,DME,15402
4,PG0224,SVO,AER,15155


In [60]:
import pandas as pd

# Assuming your connection to the database is established and assigned to 'conn'

query = """
SELECT departure_airport, COUNT(*) AS num_flights
FROM flights
GROUP BY departure_airport
ORDER BY num_flights DESC
LIMIT 5;
"""

popular_departures = pd.read_sql(query, connection)

print("Top 5 Most Frequent Departure Airports:")
(popular_departures)


Top 5 Most Frequent Departure Airports:


,departure_airport,num_flights
0,DME,3217
1,SVO,2981
2,LED,1900
3,VKO,1719
4,OVB,1055


In [63]:
import pandas as pd

# Assuming your connection to the database is established and assigned to 'conn'

query = """
SELECT arrival_airport, COUNT(*) AS num_flights
FROM flights
GROUP BY arrival_airport
ORDER BY num_flights DESC
LIMIT 5;
"""

popular_arrivals = pd.read_sql(query, connection)

print("Top 5 Most Frequent Arrival Airports:")
(popular_arrivals)


Top 5 Most Frequent Arrival Airports:


,arrival_airport,num_flights
0,DME,3217
1,SVO,2982
2,LED,1902
3,VKO,1717
4,OVB,1055


In [67]:
import pandas as pd

# Assuming your connection to the database is established and assigned to 'conn'

query_all_airports = """
SELECT departure_airport, COUNT(*) AS num_flights
FROM flights
GROUP BY departure_airport;
"""

all_airports = pd.read_sql(query_all_airports, connection)

# Assuming at least 5 airports exist (modify limit otherwise)
least_busy_airports = all_airports.sort_values(by=['num_flights']).head(5)

print("List of 5 Least Busy Departure Airports:")
least_busy_airports


List of 5 Least Busy Departure Airports:


,departure_airport,num_flights
94,USK,18
39,KXK,18
68,PKC,26
58,NYA,27
70,PYJ,27


In [72]:
import pandas as pd

# Assuming your connection to the database is established and assigned to 'conn'

query = """
SELECT a.model, COUNT(DISTINCT f.flight_id) AS num_flights
FROM flights AS f
JOIN aircrafts_data AS a ON f.aircraft_code = a.aircraft_code
GROUP BY a.model
ORDER BY num_flights DESC
LIMIT 5;
"""

popular_models = pd.read_sql(query, connection)

print("Top 5 Most Used Aircraft Models:")
(popular_models)



Top 5 Most Used Aircraft Models:


,model,num_flights
0,"{""en"": ""Cessna 208 Caravan"", ""ru"": ""Сессна 208...",9273
1,"{""en"": ""Bombardier CRJ-200"", ""ru"": ""Бомбардье ...",9048
2,"{""en"": ""Sukhoi Superjet-100"", ""ru"": ""Сухой Суп...",8504
3,"{""en"": ""Airbus A321-200"", ""ru"": ""Аэробус A321-...",1952
4,"{""en"": ""Boeing 737-300"", ""ru"": ""Боинг 737-300""}",1274


In [76]:
import pandas as pd

# Assuming your connection to the database is established and assigned to 'conn'

query = """
SELECT DATE(book_date) AS booking_date, COUNT(*) AS num_bookings
FROM bookings
GROUP BY DATE(book_date)
ORDER BY num_bookings DESC
LIMIT 5;
"""

day_with_most_bookings = pd.read_sql(query, connection)
day_with_most_bookings


,booking_date,num_bookings
0,None,262788


In [83]:
import pandas as pd

# Assuming your connection to the database is established and assigned to 'conn'

query = """
SELECT f.departure_airport || '-' || f.arrival_airport AS route, 
       SUM(t.fare * b.num_passengers) AS total_revenue,
       SUM(b.num_passengers) AS total_passengers
FROM tickets AS t
JOIN bookings AS b ON t.ticket_no = b.book_ref
JOIN ticket_flights AS tf ON tf.ticket_no = b.book_ref
JOIN flights AS f ON tf.flight_id = f.flight_id
GROUP BY f.departure_airport || '-' || f.arrival_airport
ORDER BY route;
"""

revenue_data = pd.read_sql(query, connection)

if revenue_data.empty:
  print(f"No booking data found.")

revenue_data["avg_revenue_per_passenger"] = revenue_data["total_revenue"] / revenue_data["total_passengers"]

print("Average Revenue per Passenger (Grouped by Route):")
print(revenue_data[["route", "avg_revenue_per_passenger"]])


DatabaseError: Execution failed on sql '
SELECT f.departure_airport || '-' || f.arrival_airport AS route, 
       SUM(t.fare * b.num_passengers) AS total_revenue,
       SUM(b.num_passengers) AS total_passengers
FROM tickets AS t
JOIN bookings AS b ON t.ticket_no = b.book_ref
JOIN ticket_flights AS tf ON tf.ticket_no = b.book_ref
JOIN flights AS f ON tf.flight_id = f.flight_id
GROUP BY f.departure_airport || '-' || f.arrival_airport
ORDER BY route;
': no such column: t.fare

In [84]:
bookings

,book_ref,book_date,total_amount
0,00000F,2017-07-05 03:12:00+03,265700
1,000012,2017-07-14 09:02:00+03,37900
2,000068,2017-08-15 14:27:00+03,18100
3,000181,2017-08-10 13:28:00+03,131800
4,0002D8,2017-08-07 21:40:00+03,23600
...,...,...,...
262783,FFFEF3,2017-07-17 07:23:00+03,56000
262784,FFFF2C,2017-08-08 05:55:00+03,10800
262785,FFFF43,2017-07-20 20:42:00+03,78500
262786,FFFFA8,2017-08-08 04:45:00+03,28800


In [85]:
tickets

,ticket_no,book_ref,passenger_id
0,0005432000987,06B046,8149 604011
1,0005432000988,06B046,8499 420203
2,0005432000989,E170C3,1011 752484
3,0005432000990,E170C3,4849 400049
4,0005432000991,F313DD,6615 976589
...,...,...,...
366728,0005435999869,D730BA,0474 690760
366729,0005435999870,D730BA,6535 751108
366730,0005435999871,A1AD46,1596 156448
366731,0005435999872,7B6A53,9374 822707


In [20]:

flights.columns

Index(['flight_id', 'flight_no', 'scheduled_departure', 'scheduled_arrival',
       'departure_airport', 'arrival_airport', 'status', 'aircraft_code',
       'actual_departure', 'actual_arrival'],
      dtype='object')

In [21]:
aircrafts_data.columns

Index(['aircraft_code', 'model', 'range'], dtype='object')

In [23]:
for n in [aircrafts_data, airports_data, boarding_passes, bookings, 
          flights, seats, ticket_flights, tickets]:
    print(f": {n.columns}")

: Index(['aircraft_code', 'model', 'range'], dtype='object')
: Index(['airport_code', 'airport_name', 'city', 'coordinates', 'timezone'], dtype='object')
: Index(['ticket_no', 'flight_id', 'boarding_no', 'seat_no'], dtype='object')
: Index(['book_ref', 'book_date', 'total_amount'], dtype='object')
: Index(['flight_id', 'flight_no', 'scheduled_departure', 'scheduled_arrival',
       'departure_airport', 'arrival_airport', 'status', 'aircraft_code',
       'actual_departure', 'actual_arrival'],
      dtype='object')
: Index(['aircraft_code', 'seat_no', 'fare_conditions'], dtype='object')
: Index(['ticket_no', 'flight_id', 'fare_conditions', 'amount'], dtype='object')
: Index(['ticket_no', 'book_ref', 'passenger_id'], dtype='object')


1. seats and aircrafts_data and flight: aircraft_code
2. airport_code in airports_data: related to dep and arr airport of flight
3. 

In [28]:
airports_data

,airport_code,airport_name,city,coordinates,timezone
0,YKS,"{""en"": ""Yakutsk Airport"", ""ru"": ""Якутск""}","{""en"": ""Yakutsk"", ""ru"": ""Якутск""}","(129.77099609375,62.0932998657226562)",Asia/Yakutsk
1,MJZ,"{""en"": ""Mirny Airport"", ""ru"": ""Мирный""}","{""en"": ""Mirnyj"", ""ru"": ""Мирный""}","(114.03900146484375,62.534698486328125)",Asia/Yakutsk
2,KHV,"{""en"": ""Khabarovsk-Novy Airport"", ""ru"": ""Хабар...","{""en"": ""Khabarovsk"", ""ru"": ""Хабаровск""}","(135.18800354004,48.5279998779300001)",Asia/Vladivostok
3,PKC,"{""en"": ""Yelizovo Airport"", ""ru"": ""Елизово""}","{""en"": ""Petropavlovsk"", ""ru"": ""Петропавловск-К...","(158.453994750976562,53.1679000854492188)",Asia/Kamchatka
4,UUS,"{""en"": ""Yuzhno-Sakhalinsk Airport"", ""ru"": ""Хом...","{""en"": ""Yuzhno-Sakhalinsk"", ""ru"": ""Южно-Сахали...","(142.718002319335938,46.8886985778808594)",Asia/Sakhalin
...,...,...,...,...,...
99,MMK,"{""en"": ""Murmansk Airport"", ""ru"": ""Мурманск""}","{""en"": ""Murmansk"", ""ru"": ""Мурманск""}","(32.7508010864257812,68.7817001342773438)",Europe/Moscow
100,ABA,"{""en"": ""Abakan Airport"", ""ru"": ""Абакан""}","{""en"": ""Abakan"", ""ru"": ""Абакан""}","(91.3850021362304688,53.7400016784667969)",Asia/Krasnoyarsk
101,BAX,"{""en"": ""Barnaul Airport"", ""ru"": ""Барнаул""}","{""en"": ""Barnaul"", ""ru"": ""Барнаул""}","(83.5384979248046875,53.363800048828125)",Asia/Krasnoyarsk
102,AAQ,"{""en"": ""Anapa Vityazevo Airport"", ""ru"": ""Витяз...","{""en"": ""Anapa"", ""ru"": ""Анапа""}","(37.3473014831539984,45.002101898192997)",Europe/Moscow


In [24]:
aircrafts_data

,aircraft_code,model,range
0,773,"{""en"": ""Boeing 777-300"", ""ru"": ""Боинг 777-300""}",11100
1,763,"{""en"": ""Boeing 767-300"", ""ru"": ""Боинг 767-300""}",7900
2,SU9,"{""en"": ""Sukhoi Superjet-100"", ""ru"": ""Сухой Суп...",3000
3,320,"{""en"": ""Airbus A320-200"", ""ru"": ""Аэробус A320-...",5700
4,321,"{""en"": ""Airbus A321-200"", ""ru"": ""Аэробус A321-...",5600
5,319,"{""en"": ""Airbus A319-100"", ""ru"": ""Аэробус A319-...",6700
6,733,"{""en"": ""Boeing 737-300"", ""ru"": ""Боинг 737-300""}",4200
7,CN1,"{""en"": ""Cessna 208 Caravan"", ""ru"": ""Сессна 208...",1200
8,CR2,"{""en"": ""Bombardier CRJ-200"", ""ru"": ""Бомбардье ...",2700


In [25]:
seats

,aircraft_code,seat_no,fare_conditions
0,319,2A,Business
1,319,2C,Business
2,319,2D,Business
3,319,2F,Business
4,319,3A,Business
...,...,...,...
1334,773,48H,Economy
1335,773,48K,Economy
1336,773,49A,Economy
1337,773,49C,Economy


In [26]:
flights

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,1185,PG0134,2017-09-10 09:50:00+03,2017-09-10 14:55:00+03,DME,BTK,Scheduled,319,\N,\N
1,3979,PG0052,2017-08-25 14:50:00+03,2017-08-25 17:35:00+03,VKO,HMA,Scheduled,CR2,\N,\N
2,4739,PG0561,2017-09-05 12:30:00+03,2017-09-05 14:15:00+03,VKO,AER,Scheduled,763,\N,\N
3,5502,PG0529,2017-09-12 09:50:00+03,2017-09-12 11:20:00+03,SVO,UFA,Scheduled,763,\N,\N
4,6938,PG0461,2017-09-04 12:25:00+03,2017-09-04 13:20:00+03,SVO,ULV,Scheduled,SU9,\N,\N
...,...,...,...,...,...,...,...,...,...,...
33116,33117,PG0063,2017-08-02 19:25:00+03,2017-08-02 20:10:00+03,SKX,SVO,Arrived,CR2,2017-08-02 19:25:00+03,2017-08-02 20:10:00+03
33117,33118,PG0063,2017-07-28 19:25:00+03,2017-07-28 20:10:00+03,SKX,SVO,Arrived,CR2,2017-07-28 19:30:00+03,2017-07-28 20:15:00+03
33118,33119,PG0063,2017-09-08 19:25:00+03,2017-09-08 20:10:00+03,SKX,SVO,Scheduled,CR2,\N,\N
33119,33120,PG0063,2017-08-01 19:25:00+03,2017-08-01 20:10:00+03,SKX,SVO,Arrived,CR2,2017-08-01 19:26:00+03,2017-08-01 20:12:00+03


In [29]:
tickets

,ticket_no,book_ref,passenger_id
0,0005432000987,06B046,8149 604011
1,0005432000988,06B046,8499 420203
2,0005432000989,E170C3,1011 752484
3,0005432000990,E170C3,4849 400049
4,0005432000991,F313DD,6615 976589
...,...,...,...
366728,0005435999869,D730BA,0474 690760
366729,0005435999870,D730BA,6535 751108
366730,0005435999871,A1AD46,1596 156448
366731,0005435999872,7B6A53,9374 822707


In [30]:
boarding_passes

,ticket_no,flight_id,boarding_no,seat_no
0,0005435212351,30625,1,2D
1,0005435212386,30625,2,3G
2,0005435212381,30625,3,4H
3,0005432211370,30625,4,5D
4,0005435212357,30625,5,11A
...,...,...,...,...
579681,0005434302871,19945,85,20F
579682,0005432892791,19945,86,21C
579683,0005434302869,19945,87,20E
579684,0005432802476,19945,88,21F
